##  Overview

This notebook presents a  analysis of apartment listings in Barcelona using cleaned and structured data from the Idealista dataset  
Our goal is:

1. **Descriptive Analysis** - Understand pricing, size, and spatial dynamics across neighborhoods and districts.
2. **Decision Support** - Develop a scoring-based system to help users identify the most efficient and valuable apartments for purchase.

In [37]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import warnings

warnings.filterwarnings("ignore")

data_path = "zones/exploitation_zone/idealista"

try:
    df = pd.read_parquet(data_path)
    print(f"Loaded data with {df.shape[0]} rows and {df.shape[1]} columns.")
except Exception as e:
    print(f"Error loading data: {e}")

df.head()


Loaded data with 19881 rows and 8 columns.


,district,neighborhood,price,size,rooms,bathrooms,hasLift,calculatedPricePerArea
0,Centre,Centre,625000.0,245.0,4,5,None,2551.0
1,Centre,Centre,180000.0,166.0,2,2,False,1084.0
2,Centre,Sant Josep,259900.0,110.0,4,2,True,2363.0
3,Centre,Carretera del Mig,164900.0,77.0,3,1,True,2142.0
4,Centre,Centre,380000.0,110.0,3,2,True,3455.0


In [38]:
df.info()
print(df.describe().T)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19881 entries, 0 to 19880
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   district                19881 non-null  object 
 1   neighborhood            12259 non-null  object 
 2   price                   19881 non-null  float64
 3   size                    19881 non-null  float64
 4   rooms                   19881 non-null  int32  
 5   bathrooms               19881 non-null  int32  
 6   hasLift                 16906 non-null  object 
 7   calculatedPricePerArea  19881 non-null  float64
dtypes: float64(3), int32(2), object(3)
memory usage: 1.1+ MB
                          count           mean            std      min  \
price                   19881.0  450008.612394  590393.277819  39000.0   
size                    19881.0     125.017011     129.489192     21.0   
rooms                   19881.0       3.199638       1.174739      0.0   
bath

In [39]:
import plotly.subplots as sp
import plotly.graph_objects as go

num_cols = df.select_dtypes(include="number").columns.tolist()

df_plot = df.copy()
df_plot = df_plot[df_plot["calculatedPricePerArea"] < 15000]

n = len(num_cols)
cols = 3
rows = (n // cols) + 1

fig = sp.make_subplots(rows=rows, cols=cols, subplot_titles=num_cols)

row = col = 1
for i, colname in enumerate(num_cols):
    fig.add_trace(
        go.Histogram(x=df_plot[colname], nbinsx=40, name=colname),
        row=row,
        col=col
    )
    col += 1
    if col > cols:
        row += 1
        col = 1

fig.update_layout(
    title_text="Distribution Overview of Numeric Features",
    height=300 * rows,
    showlegend=False
)

fig.show()


- **Price Distribution:**  
  The mean apartment price is ~€448k, but the high standard deviation and a maximum of €12 M indicate a wide spread, with luxury listings skewing the upper tail.

- **Size Distribution:**  
  Average size is ~123 m², with 75% of apartments under 125 m². A handful of large units (up to 1,000 m²) remain after outlier removal.

- **Room & Bathroom Counts:**  
  Most apartments have 3 rooms and 1 bathroom, reflecting typical urban layouts. A small portion has 0 rooms or 0 bathrooms—these may be studio-like or data quirks.

- **Elevator Availability:**  
  With an average of 0.62, about 62% of listings include an elevator, highlighting accessibility considerations for the remaining 38%.

- **Price per m²:**  
  The average rate is ~€3,365/m², but ranges from under €500/m² to over €17,000/m² in premium areas.

In [40]:
neigh_price = (
    df.groupby("neighborhood")["calculatedPricePerArea"]
      .mean()
      .sort_values(ascending=False)
      .reset_index()
)
neigh_price.head(10)

,neighborhood,calculatedPricePerArea
0,La Dreta de l'Eixample,6878.617284
1,Les Tres Torres,6371.761905
2,Pedralbes,6331.714286
3,Sant Gervasi - La Bonanova,6302.740476
4,L'Antiga Esquerra de l'Eixample,6074.937500
5,Sant Pere - Santa Caterina i la Ribera,5831.000000
6,Les Corts,5797.313131
7,Sant Gervasi - Galvany,5785.721277
8,Sarrià,5760.562791
9,El Gòtic,5022.875000


In [41]:
neigh_size = (
    df.groupby("neighborhood")["size"]
      .mean()
      .sort_values(ascending=False)
      .reset_index()
)
neigh_size.head(10)

,neighborhood,size
0,Vallvidrera - El Tibidabo i les Planes,318.357513
1,Sarrià,304.102326
2,Pedralbes,275.073892
3,Sant Gervasi - La Bonanova,267.269048
4,Les Tres Torres,197.435374
5,Sant Gervasi - Galvany,186.974468
6,La Dreta de l'Eixample,171.574074
7,La Font d'En Fargues,167.125000
8,Sant Andreu,165.000000
9,Vila de Gràcia,162.085526


### Observations
- **Pedralbes** and **Sant Gervasi – La Bonanova** rank high both in price and size, indicating luxury large‐unit markets.  
- **La Dreta de l’Eixample** has the highest price tag per m² but only mid‐range sizes, suggesting a cost premium for central location rather than square footage.  

In [42]:
top8 = neigh_price.head(8)["neighborhood"].tolist()

subset = df[df["neighborhood"].isin(top8)]

bar = subset.groupby("neighborhood")["calculatedPricePerArea"].mean().reindex(top8)
line = subset.groupby("neighborhood")["size"].mean().reindex(top8)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=bar.index,
    y=bar.values,
    name="Avg Price (€ / m²)"
))

fig.add_trace(go.Scatter(
    x=line.index,
    y=line.values,
    mode="lines+markers",
    name="Avg Size (m²)",
    yaxis="y2"
))

fig.update_layout(
    title="Top 8 Neighborhoods: Avg Price vs. Avg Size",
    xaxis_tickangle=-45,
    yaxis=dict(title="Avg Price (€ / m²)"),
    yaxis2=dict(
        title="Avg Size (m²)",
        overlaying="y",
        side="right"
    ),
    legend=dict(x=0.1, y=1.1)
)

fig.show()

## Price vs. Size Trade-Offs

- **La Dreta de l’Eixample** and **Les Tres Torres** top the price chart (> €6 000/m²) but offer only mid-range apartment sizes (~180–200 m²). This suggests a strong location premium rather than extra square footage.  
- **Pedralbes** and **Sant Gervasi – La Bonanova** combine both high price and the largest average sizes (~260–280 m²), indicating a true luxury segment where buyers pay top dollar for genuinely spacious units.  
- **L’Antiga Esquerra de l’Eixample** and **Sant Pere – Santa Caterina i la Ribera** share similar price points (~€6 000/m²) but diverge on size: the former averages smaller apartments (~130 m²) while the latter is slightly larger (~140 m²), offering a marginally better space-for-money ratio.  
- **Les Corts** and **Sant Gervasi – Galvany** sit in the mid-tier price bracket (€5 800–6 000/m²) with modest sizes (~100–120 m²), making them potential “value” options for those prioritizing central addresses over maximum area.  

**Key Insight:**  
- If **maximum size** is the priority, **Pedralbes** and **Sant Gervasi – La Bonanova** are clear winners, albeit at luxury prices.  
- If **cost efficiency** (more m² per €) matters, neighborhoods like **Les Corts** and **Sant Pere – Santa Caterina i la Ribera** may strike a better balance.  
- Central hotspot **La Dreta de l’Eixample** demands a premium price without delivering the largest units—best suited for buyers who value location prestige above all.  

In [43]:
subset["area_per_100k"] = 100_000 / subset["calculatedPricePerArea"]


# Считаем среднее по району
area_100k = subset.groupby("neighborhood")["area_per_100k"].mean().reindex(top8)
size = subset.groupby("neighborhood")["size"].mean().reindex(top8)

# Создаём график
fig = go.Figure()

# Bar trace: m² per €100k (value-for-money)
fig.add_trace(go.Bar(
    x=area_100k.index,
    y=area_100k.values,
    name="m² per €100k",
    marker_color="darkgreen"
))

# Line trace: Avg apartment size
fig.add_trace(go.Scatter(
    x=size.index,
    y=size.values,
    mode="lines+markers",
    name="Avg Size (m²)",
    yaxis="y2"
))

# Layout
fig.update_layout(
    title="Top 8 Neighborhoods: Value for Money vs. Avg Size",
    xaxis_tickangle=-45,
    yaxis=dict(title="m² per €100k"),
    yaxis2=dict(
        title="Avg Size (m²)",
        overlaying="y",
        side="right"
    ),
    legend=dict(x=0.1, y=1.1)
)

fig.show()

In [44]:
agg = (
    subset.groupby("neighborhood")
          .agg(
              avg_price_m2=("calculatedPricePerArea", "mean"),
              avg_size=("size", "mean"),
              area_per_100k=("area_per_100k", "mean")
          )
          .reindex(top8)
          .reset_index()
)

fig = px.scatter(
    agg,
    x="avg_price_m2",
    y="avg_size",
    size="area_per_100k",
    color="neighborhood",
    text="neighborhood",
    size_max=60,
    title="Top 8 Neighborhoods: Price vs Size vs Value",
    labels={
        "avg_price_m2": "Avg Price per m² (€)",
        "avg_size": "Avg Size (m²)",
        "area_per_100k": "m² per €100k"
    }
)

fig.update_traces(textposition='top center')
fig.update_layout(
    xaxis_tickformat=',',
    yaxis_tickformat=',',
    xaxis=dict(title="Avg Price per m² (€)"),
    yaxis=dict(title="Avg Size (m²)"),
    legend_title_text="Neighborhood"
)

fig.show()

## Value-for-Money Landscape

* **La Dreta de l’Eixample** remains the most expensive neighborhood (\~€6 900/m²), yet its average apartment size (\~170 m²) and relatively small bubble suggest poor value-for-money — high prestige, but not generous space per euro.
* **Pedralbes** and **Sant Gervasi – La Bonanova** sit in the upper-right quadrant, offering **both** large units (\~270–280 m²) and higher value than La Dreta, despite luxury prices — ideal for affluent buyers seeking maximum comfort.
* **Les Tres Torres**, while nearly matching Pedralbes in price, offers significantly less value — smaller average size (\~200 m²) and lower space return per €100k.
* **Les Corts** and **Sant Pere – Santa Caterina i la Ribera** appear as **budget-friendly options**: they feature **modest prices (\~€5 800/m²)** and relatively high m²-per-€ ratios, making them ideal for buyers focused on cost efficiency.
* **Sant Gervasi – Galvany** stands out as a **balanced performer** — combining large average sizes (\~240 m²) with below-average prices and the **largest value bubble** in the chart.

**Key Insight:**

* For **best value** (most m² per €100k), **Sant Gervasi – Galvany** and **Les Corts** outperform more glamorous areas like **Les Tres Torres** and **La Dreta de l’Eixample**.
* **Pedralbes** offers a compelling trade-off for buyers with high budgets: it's expensive, yes, but delivers on spaciousness and relative value.
* **La Dreta de l’Eixample** is a clear outlier: buyers here pay a location premium, not necessarily for comfort or square meters.

In [45]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

agg = (
    df.groupby("neighborhood")
      .agg(
          avg_price_per_m2=("calculatedPricePerArea", "mean"),
          avg_size=("size", "mean")
      )
      .copy()
)

agg["area_per_100k"] = 100_000 / agg["avg_price_per_m2"]

scaler = StandardScaler()
scaled_features = scaler.fit_transform(agg)

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
agg["cluster"] = kmeans.fit_predict(scaled_features)

agg_plot = agg.reset_index()

import plotly.express as px

fig = px.scatter(
    agg_plot,
    x="avg_price_per_m2",
    y="avg_size",
    size="area_per_100k",
    color="cluster",
    hover_name="neighborhood",
    title="Neighborhood Clustering: Price vs. Size vs. Value",
    labels={
        "avg_price_per_m2": "Avg Price per m² (€)",
        "avg_size": "Avg Size (m²)",
        "area_per_100k": "m² per €100k",
        "cluster": "Cluster"
    },
    size_max=50
)

fig.show()


## Neighborhood Clusters: Price, Size & Value

We applied k-means clustering to group neighborhoods based on their **average price per square meter**, **average apartment size**, and **value-for-money** (m² per €100k). The visualization reveals three distinct market segments:

* **Cluster 0 (Dark Blue): Budget Segment**
  These neighborhoods offer the **lowest average prices** (often < €3 000/m²) and **relatively smaller apartments**. However, their **large bubble sizes** reflect **strong value-for-money**, making them appealing to price-sensitive buyers or investors seeking rental yields. This segment dominates the left side of the plot.

* **Cluster 1 (Yellow): Mid-Market Trade-Offs**
  This group occupies the central region — with moderate prices (€3 000–5 000/m²) and mid-sized apartments (\~100–150 m²). Many of these areas balance affordability with comfort, but exhibit varied value returns (smaller bubbles), suggesting that buyers need to **cherry-pick** within this group.

* **Cluster 2 (Red-Pink): Premium Segment**
  Representing the **highest priced neighborhoods** (> €5 500/m²), these areas offer **larger apartments**, often exceeding 200 m². Despite their space, **smaller bubble sizes** indicate lower value-for-money — buyers here pay for exclusivity and location prestige rather than efficiency.

**Key Insight:**

* **Cluster 0** is ideal for **cost-conscious buyers or investors**, maximizing space per euro.
* **Cluster 1** is a **mixed bag**, suitable for **pragmatic buyers** willing to trade off certain features.
* **Cluster 2** targets **luxury buyers** prioritizing **status, location, and comfort**, often at the cost of efficiency.

_______________________

## From Market Analysis to Individual Listings

Having analyzed average prices, sizes, and value indicators across neighborhoods, we now shift focus to the **apartment level**.

Our goal:  
To design a scoring mechanism that allows us to rank individual listings based on multiple value-relevant dimensions, such as cost, space, and amenities.

The following methodology describes how the score is built and used to identify the most desirable apartments.


## Scoring Methodology

To rank apartment listings by overall efficiency and desirability, we construct a composite score based on key features affecting cost, space, and livability.

### 1. Selected Features

| Feature                  | Contribution | Reason                            |
| ------------------------ | ------------ | --------------------------------- |
| `size` (m²)              | Positive     | More space = higher comfort       |
| `price` (€)              | Negative     | Lower cost = higher affordability |
| `calculatedPricePerArea` | Negative     | Lower €/m² = better value         |
| `rooms`                  | Positive     | More layout flexibility           |
| `hasLift`                | Positive     | Improved accessibility            |

Listings with extreme values (e.g. > €10M or > 1000 m²) are excluded.

### 2. Normalization & Inversion

All features are **Min-Max scaled** to \[0, 1].
Negative contributors (price, €/m²) are inverted:

$$
\text{inv\_x} = 1 - x_{\text{scaled}}
$$


### 3. Score Composition

Final score is a weighted sum:

$$
\text{score} = 0.3 \cdot \text{size} + 0.2 \cdot \text{inv\_price} + 0.2 \cdot \text{inv\_€/m²} + 0.2 \cdot \text{rooms} + 0.1 \cdot \text{hasLift}
$$


### 4. Output

Listings are sorted by score ∈ \[0, 1], where higher scores reflect better overall value based on the defined criteria.

In [62]:
from sklearn.preprocessing import MinMaxScaler

# ─────────────── Step 1: Filter sensible listings ───────────────
filtered = df[
    (df["size"] >= 60) &
    (df["price"] <= 800_000) &
    (df["calculatedPricePerArea"] <= 7000)
].copy()

# ─────────────── Step 2: Normalize and invert features ───────────────
features = ["size", "price", "calculatedPricePerArea", "rooms"]
scaler = MinMaxScaler()
filtered[features] = scaler.fit_transform(filtered[features])

filtered["inv_price"] = 1 - filtered["price"]
filtered["inv_price_per_m2"] = 1 - filtered["calculatedPricePerArea"]
filtered["hasLift"] = filtered["hasLift"].fillna(False).astype(int)

# ─────────────── Step 3: Compute composite score ───────────────
filtered["score"] = (
    0.3 * filtered["size"] +
    0.2 * filtered["inv_price"] +
    0.2 * filtered["inv_price_per_m2"] +
    0.2 * filtered["rooms"] +
    0.1 * filtered["hasLift"]
)

# ─────────────── Step 4: Output top candidates ───────────────
top_k = (
    filtered.sort_values("score", ascending=False)
    .loc[:, ["neighborhood", "district", "price", "size", "rooms", "bathrooms", "calculatedPricePerArea", "hasLift", "score"]]
    .head(500)
)

top_k


,neighborhood,district,price,size,rooms,bathrooms,calculatedPricePerArea,hasLift,score
9120,None,Marianao,0.954008,1.000000,0.363636,3,0.000000,0,0.581926
18722,None,Marianao,0.954008,1.000000,0.363636,3,0.000000,0,0.581926
9353,None,Marianao,0.954008,1.000000,0.363636,3,0.000000,0,0.581926
3365,El Poble Sec - Parc de Montjuïc,Sants-Montjuïc,0.000000,0.011834,0.363636,2,0.007771,1,0.574723
13491,None,Camps Blancs - Casablanca - Canons,0.558476,0.317751,1.000000,3,0.051806,0,0.573269
...,...,...,...,...,...,...,...,...,...
6701,Can Serra,Can Serra - Pubilla Cases,0.190539,0.011834,0.363636,1,0.283864,1,0.481397
6676,Can Serra,Can Serra - Pubilla Cases,0.190539,0.011834,0.363636,1,0.283864,1,0.481397
12360,Can Serra,Can Serra - Pubilla Cases,0.190539,0.011834,0.363636,1,0.283864,1,0.481397
6718,Can Serra,Can Serra - Pubilla Cases,0.190539,0.011834,0.363636,1,0.283864,1,0.481397


In [63]:
min_max_dict = {}
for col in ["price", "size", "calculatedPricePerArea"]:
    col_min = df[df["size"] >= 60][col].min()
    col_max = df[df["size"] >= 60][col].max()
    min_max_dict[col] = (col_min, col_max)

def reverse_minmax(val, col):
    min_val, max_val = min_max_dict[col]
    return val * (max_val - min_val) + min_val

top_k["price (€)"] = top_k["price"].apply(lambda x: reverse_minmax(x, "price")).round(0)
top_k["size (m²)"] = top_k["size"].apply(lambda x: reverse_minmax(x, "size")).round(1)
top_k["€/m²"] = top_k["calculatedPricePerArea"].apply(lambda x: reverse_minmax(x, "calculatedPricePerArea")).round(1)

top_k

,neighborhood,district,price,size,rooms,bathrooms,calculatedPricePerArea,hasLift,score,price (€),size (m²),€/m²
9120,None,Marianao,0.954008,1.000000,0.363636,3,0.000000,0,0.581926,11449888.0,3330.0,287.0
18722,None,Marianao,0.954008,1.000000,0.363636,3,0.000000,0,0.581926,11449888.0,3330.0,287.0
9353,None,Marianao,0.954008,1.000000,0.363636,3,0.000000,0,0.581926,11449888.0,3330.0,287.0
3365,El Poble Sec - Parc de Montjuïc,Sants-Montjuïc,0.000000,0.011834,0.363636,2,0.007771,1,0.574723,39000.0,98.7,418.0
13491,None,Camps Blancs - Casablanca - Canons,0.558476,0.317751,1.000000,3,0.051806,0,0.573269,6718928.0,1099.0,1160.2
...,...,...,...,...,...,...,...,...,...,...,...,...
6701,Can Serra,Can Serra - Pubilla Cases,0.190539,0.011834,0.363636,1,0.283864,1,0.481397,2318034.0,98.7,5071.8
6676,Can Serra,Can Serra - Pubilla Cases,0.190539,0.011834,0.363636,1,0.283864,1,0.481397,2318034.0,98.7,5071.8
12360,Can Serra,Can Serra - Pubilla Cases,0.190539,0.011834,0.363636,1,0.283864,1,0.481397,2318034.0,98.7,5071.8
6718,Can Serra,Can Serra - Pubilla Cases,0.190539,0.011834,0.363636,1,0.283864,1,0.481397,2318034.0,98.7,5071.8


In [65]:
top_k["label"] = (
    top_k["neighborhood"].fillna("Unknown") + " | " + top_k["district"].astype(str)
)

fig = px.bar(
    top_k.sort_values("score"),
    x=(top_k["score"] * 100).round(1),
    y="label",
    orientation="h",
    color="hasLift",
    text=top_k.apply(lambda row: f"{int(row['price (€)'])}€ | {row['size (m²)']}m² | {row['€/m²']}€/m²", axis=1),
    title="Top Apartments Ranked by Score",
    labels={"score": "Score", "label": "Listing (Neighborhood | District)", "hasLift": "Elevator"}
)

fig.update_traces(textposition="outside")
fig.update_layout(height=600)
fig.show()


## Summary

This analysis examined Barcelona’s housing market

Key insights:

- **Price vs. Size Trade-Offs** clearly differentiate luxury zones (Pedralbes, Bonanova) from cost-efficient areas (Les Corts, Galvany).
- **Value-for-money metrics** reveal that centrality often comes at the cost of space and efficiency.
- **Clustering** identifies three market segments: premium, mid-range, and budget zones.
- The **scoring system** surfaces highly efficient listings — especially in outer but well-connected areas such as **Can Serra**, **Marianao**, **Bellvitge**, and **La Gavarra**.
- While some top listings lack detailed neighborhood labels, their price-performance ratios make them stand out.
